# Lab 3: Constraint Satisfaction Problems

## Learning Objectives

By the end of this lab, you will:
- Understand CSP formulation and representation
- Implement backtracking search
- Apply constraint propagation and arc consistency
- Use variable and value ordering heuristics
- Solve real CSPs (Sudoku, map coloring, scheduling)
- Know when CSPs are the right approach

## What are Constraint Satisfaction Problems?

A **CSP** consists of:
- **Variables**: X = {X₁, X₂, ..., Xₙ}
- **Domains**: D = {D₁, D₂, ..., Dₙ} (possible values for each variable)
- **Constraints**: C (restrictions on variable combinations)

**Goal**: Find an assignment of values to variables that satisfies all constraints.

## Why CSPs?

**Advantages**:
- Natural problem representation
- General-purpose algorithms
- Powerful inference techniques
- Can prove infeasibility

**Key Insight**: Use problem structure to prune search space!

## Real-World Applications

- 📅 **Scheduling**: Class timetables, shift assignments
- 🎨 **Map Coloring**: Frequency assignment, register allocation
- 🧩 **Puzzles**: Sudoku, crosswords, n-queens
- 🏭 **Resource Allocation**: Manufacturing, logistics
- 🧬 **Bioinformatics**: Protein structure prediction


In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Set, Tuple, Optional, Callable
from collections import deque
from copy import deepcopy
import time

# Set random seed
np.random.seed(42)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Part 1: CSP Basics - Map Coloring

### Classic Example: Australia Map Coloring

**Problem**: Color map regions so no adjacent regions have the same color.

**Variables**: {WA, NT, SA, Q, NSW, V, T}
**Domains**: {Red, Green, Blue} for each variable
**Constraints**: Adjacent regions must have different colors


In [ ]:
class CSP:
    """Generic Constraint Satisfaction Problem framework."""
    
    def __init__(self, variables: List[str], domains: Dict[str, List],
                 constraints: List[Tuple]):
        """
        Initialize CSP.
        
        Args:
            variables: List of variable names
            domains: Dict mapping variable to list of possible values
            constraints: List of (var1, var2) pairs that must differ
        """
        self.variables = variables
        self.domains = {var: list(domain) for var, domain in domains.items()}
        self.constraints = constraints
        
        # Build constraint graph (neighbors)
        self.neighbors = {var: [] for var in variables}
        for var1, var2 in constraints:
            self.neighbors[var1].append(var2)
            self.neighbors[var2].append(var1)
    
    def is_consistent(self, var: str, value, assignment: Dict[str, any]) -> bool:
        """Check if assigning value to var is consistent with assignment."""
        for neighbor in self.neighbors[var]:
            if neighbor in assignment and assignment[neighbor] == value:
                return False
        return True
    
    def is_complete(self, assignment: Dict[str, any]) -> bool:
        """Check if assignment is complete (all variables assigned)."""
        return len(assignment) == len(self.variables)
    
    def select_unassigned_variable(self, assignment: Dict[str, any]) -> Optional[str]:
        """Select next variable to assign (simple version)."""
        for var in self.variables:
            if var not in assignment:
                return var
        return None


# Australia map coloring
print("Map Coloring Problem: Australia")
print("=" * 60)

# Define the problem
variables = ['WA', 'NT', 'SA', 'Q', 'NSW', 'V', 'T']

# Each region can be Red, Green, or Blue
colors = ['Red', 'Green', 'Blue']
domains = {var: colors for var in variables}

# Adjacent regions (constraints)
constraints = [
    ('WA', 'NT'), ('WA', 'SA'),
    ('NT', 'SA'), ('NT', 'Q'),
    ('SA', 'Q'), ('SA', 'NSW'), ('SA', 'V'),
    ('Q', 'NSW'),
    ('NSW', 'V')
]

australia = CSP(variables, domains, constraints)

print(f"Variables: {len(variables)}")
print(f"Domain size: {len(colors)}")
print(f"Constraints: {len(constraints)}")
print(f"Search space: {len(colors)**len(variables):,} possible assignments")
print()
print("Neighbors (adjacent regions):")
for var in sorted(variables):
    print(f"  {var}: {australia.neighbors[var]}")

## Part 2: Backtracking Search

### Basic Backtracking Algorithm

**Algorithm**:
```
function BACKTRACK(assignment, csp):
    if assignment is complete:
        return assignment
    
    var = SELECT-UNASSIGNED-VARIABLE(csp)
    
    for value in DOMAIN-VALUES(var, csp):
        if value is consistent with assignment:
            add {var = value} to assignment
            result = BACKTRACK(assignment, csp)
            if result ≠ failure:
                return result
            remove {var = value} from assignment
    
    return failure
```


In [ ]:
class BacktrackingSearch:
    """Backtracking search for CSP."""
    
    def __init__(self, csp: CSP):
        self.csp = csp
        self.assignments_tried = 0
        self.backtracks = 0
    
    def solve(self) -> Optional[Dict[str, any]]:
        """Solve CSP using backtracking."""
        self.assignments_tried = 0
        self.backtracks = 0
        return self._backtrack({})
    
    def _backtrack(self, assignment: Dict[str, any]) -> Optional[Dict[str, any]]:
        """Recursive backtracking."""
        # Check if complete
        if self.csp.is_complete(assignment):
            return assignment
        
        # Select unassigned variable
        var = self.csp.select_unassigned_variable(assignment)
        
        # Try each value in domain
        for value in self.csp.domains[var]:
            self.assignments_tried += 1
            
            # Check consistency
            if self.csp.is_consistent(var, value, assignment):
                # Add to assignment
                assignment[var] = value
                
                # Recurse
                result = self._backtrack(assignment)
                if result is not None:
                    return result
                
                # Backtrack
                del assignment[var]
                self.backtracks += 1
        
        return None


# Solve Australia map coloring
print("Solving Australia Map Coloring with Backtracking")
print("=" * 60)

solver = BacktrackingSearch(australia)
start_time = time.time()
solution = solver.solve()
end_time = time.time()

if solution:
    print("✓ Solution found!\n")
    for region in sorted(solution.keys()):
        print(f"  {region:4s}: {solution[region]}")
    print()
    print(f"Statistics:")
    print(f"  Assignments tried: {solver.assignments_tried}")
    print(f"  Backtracks: {solver.backtracks}")
    print(f"  Time: {(end_time - start_time)*1000:.2f} ms")
else:
    print("✗ No solution exists.")

### Visualizing the Solution


In [ ]:
def visualize_australia_map(solution: Dict[str, str]):
    """Visualize Australia map coloring solution."""
    
    # Simple visualization using matplotlib
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Approximate positions (for visualization)
    positions = {
        'WA': (0.2, 0.5),
        'NT': (0.4, 0.7),
        'SA': (0.5, 0.4),
        'Q': (0.7, 0.7),
        'NSW': (0.8, 0.4),
        'V': (0.7, 0.2),
        'T': (0.8, 0.05)
    }
    
    # Color mapping
    color_map = {
        'Red': 'red',
        'Green': 'green',
        'Blue': 'blue'
    }
    
    # Draw regions
    for region, pos in positions.items():
        color = color_map[solution[region]]
        circle = plt.Circle(pos, 0.08, color=color, alpha=0.6, 
                           edgecolor='black', linewidth=3)
        ax.add_patch(circle)
        ax.text(pos[0], pos[1], region, ha='center', va='center',
               fontsize=14, fontweight='bold', color='white')
    
    # Draw edges (constraints)
    for var1, var2 in constraints:
        pos1 = positions[var1]
        pos2 = positions[var2]
        ax.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]], 
               'k-', alpha=0.3, linewidth=2)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Australia Map Coloring Solution', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

if solution:
    visualize_australia_map(solution)

## Part 3: Improving Backtracking with Heuristics

### Variable Ordering Heuristics

**Minimum Remaining Values (MRV)**: Choose variable with fewest legal values
- Also called "most constrained variable"
- Fail fast if no solution exists

**Degree Heuristic**: Choose variable involved in most constraints
- Tie-breaker for MRV

### Value Ordering Heuristic

**Least Constraining Value**: Try values that rule out fewest choices for neighbors
- Maximize flexibility for future assignments


In [ ]:
class ImprovedBacktracking:
    """Backtracking with MRV and LCV heuristics."""
    
    def __init__(self, csp: CSP, use_mrv: bool = True, use_lcv: bool = True):
        self.csp = csp
        self.use_mrv = use_mrv
        self.use_lcv = use_lcv
        self.assignments_tried = 0
        self.backtracks = 0
    
    def mrv_heuristic(self, assignment: Dict[str, any]) -> Optional[str]:
        """Select variable with minimum remaining values."""
        unassigned = [v for v in self.csp.variables if v not in assignment]
        
        if not unassigned:
            return None
        
        # Count legal values for each variable
        def count_legal_values(var):
            count = 0
            for value in self.csp.domains[var]:
                if self.csp.is_consistent(var, value, assignment):
                    count += 1
            return count
        
        return min(unassigned, key=count_legal_values)
    
    def lcv_ordering(self, var: str, assignment: Dict[str, any]) -> List:
        """Order values by least constraining value."""
        def count_conflicts(value):
            # Count how many values this rules out for neighbors
            conflicts = 0
            for neighbor in self.csp.neighbors[var]:
                if neighbor not in assignment:
                    for neighbor_value in self.csp.domains[neighbor]:
                        if neighbor_value == value:
                            conflicts += 1
            return conflicts
        
        values = self.csp.domains[var]
        return sorted(values, key=count_conflicts)
    
    def solve(self) -> Optional[Dict[str, any]]:
        """Solve CSP using improved backtracking."""
        self.assignments_tried = 0
        self.backtracks = 0
        return self._backtrack({})
    
    def _backtrack(self, assignment: Dict[str, any]) -> Optional[Dict[str, any]]:
        """Recursive backtracking with heuristics."""
        if self.csp.is_complete(assignment):
            return assignment
        
        # Select variable (with or without MRV)
        if self.use_mrv:
            var = self.mrv_heuristic(assignment)
        else:
            var = self.csp.select_unassigned_variable(assignment)
        
        # Order values (with or without LCV)
        if self.use_lcv:
            values = self.lcv_ordering(var, assignment)
        else:
            values = self.csp.domains[var]
        
        # Try each value
        for value in values:
            self.assignments_tried += 1
            
            if self.csp.is_consistent(var, value, assignment):
                assignment[var] = value
                
                result = self._backtrack(assignment)
                if result is not None:
                    return result
                
                del assignment[var]
                self.backtracks += 1
        
        return None


# Compare different configurations
print("Comparing Backtracking Variants")
print("=" * 60)

configs = [
    (False, False, "Basic"),
    (True, False, "MRV only"),
    (False, True, "LCV only"),
    (True, True, "MRV + LCV")
]

results = []

for use_mrv, use_lcv, name in configs:
    solver = ImprovedBacktracking(australia, use_mrv, use_lcv)
    start = time.time()
    solution = solver.solve()
    elapsed = time.time() - start
    
    results.append({
        'name': name,
        'assignments': solver.assignments_tried,
        'backtracks': solver.backtracks,
        'time': elapsed * 1000
    })
    
    print(f"{name:15s}: {solver.assignments_tried:4d} assignments, "
          f"{solver.backtracks:3d} backtracks, {elapsed*1000:.2f} ms")

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

names = [r['name'] for r in results]
assignments = [r['assignments'] for r in results]
backtracks = [r['backtracks'] for r in results]

ax1.bar(names, assignments, color='skyblue', edgecolor='navy', linewidth=2, alpha=0.7)
ax1.set_ylabel('Assignments Tried', fontweight='bold')
ax1.set_title('Efficiency: Assignments', fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=15)

ax2.bar(names, backtracks, color='lightcoral', edgecolor='darkred', linewidth=2, alpha=0.7)
ax2.set_ylabel('Backtracks', fontweight='bold')
ax2.set_title('Efficiency: Backtracks', fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=15)

plt.tight_layout()
plt.show()

print("\nHeuristics dramatically reduce search effort!")

## Part 4: Constraint Propagation and Arc Consistency

### Forward Checking

After assigning a variable, remove inconsistent values from neighbors' domains.

### Arc Consistency (AC-3)

**Algorithm**: Make every arc consistent
- Arc (X, Y) is consistent if for every value x in Domain(X), there exists a value y in Domain(Y) that satisfies the constraint

**AC-3 Algorithm**:
```
1. Add all arcs to queue
2. While queue not empty:
   - Remove arc (X, Y)
   - If REVISE(X, Y) changed Domain(X):
     - Add all arcs (Z, X) to queue (for all Z ≠ Y)
```


In [ ]:
class AC3:
    """AC-3 algorithm for arc consistency."""
    
    def __init__(self, csp: CSP):
        self.csp = csp
    
    def revise(self, xi: str, xj: str, domains: Dict[str, List]) -> bool:
        """Make xi arc-consistent with xj."""
        revised = False
        
        # Check each value in xi's domain
        for x in domains[xi][:]:
            # Check if there exists a satisfying value in xj's domain
            satisfies = False
            for y in domains[xj]:
                if x != y:  # For map coloring: different colors
                    satisfies = True
                    break
            
            # If no satisfying value, remove x
            if not satisfies:
                domains[xi].remove(x)
                revised = True
        
        return revised
    
    def enforce_arc_consistency(self, domains: Dict[str, List] = None) -> Dict[str, List]:
        """Enforce arc consistency on all arcs."""
        if domains is None:
            domains = {var: list(dom) for var, dom in self.csp.domains.items()}
        
        # Initialize queue with all arcs
        queue = deque()
        for var1, var2 in self.csp.constraints:
            queue.append((var1, var2))
            queue.append((var2, var1))
        
        # Process arcs
        while queue:
            xi, xj = queue.popleft()
            
            if self.revise(xi, xj, domains):
                # If domain is empty, no solution
                if len(domains[xi]) == 0:
                    return None
                
                # Add all arcs (xk, xi) where xk is neighbor of xi
                for xk in self.csp.neighbors[xi]:
                    if xk != xj:
                        queue.append((xk, xi))
        
        return domains


class BacktrackingWithAC3:
    """Backtracking with AC-3 constraint propagation."""
    
    def __init__(self, csp: CSP):
        self.csp = csp
        self.ac3 = AC3(csp)
        self.assignments_tried = 0
        self.backtracks = 0
    
    def solve(self) -> Optional[Dict[str, any]]:
        """Solve CSP with AC-3 preprocessing."""
        # First, enforce arc consistency
        domains = self.ac3.enforce_arc_consistency()
        
        if domains is None:
            return None
        
        print("After AC-3:")
        for var in sorted(domains.keys()):
            print(f"  {var}: {domains[var]}")
        print()
        
        self.assignments_tried = 0
        self.backtracks = 0
        return self._backtrack({}, domains)
    
    def _backtrack(self, assignment: Dict[str, any], 
                   domains: Dict[str, List]) -> Optional[Dict[str, any]]:
        """Backtracking with maintained arc consistency."""
        if len(assignment) == len(self.csp.variables):
            return assignment
        
        # Select variable with smallest domain (MRV)
        unassigned = [v for v in self.csp.variables if v not in assignment]
        var = min(unassigned, key=lambda v: len(domains[v]))
        
        for value in domains[var]:
            self.assignments_tried += 1
            
            if self.csp.is_consistent(var, value, assignment):
                assignment[var] = value
                
                # Create new domain dict with var assigned
                new_domains = {v: list(d) for v, d in domains.items()}
                new_domains[var] = [value]
                
                # Maintain arc consistency
                new_domains = self.ac3.enforce_arc_consistency(new_domains)
                
                if new_domains is not None:
                    result = self._backtrack(assignment, new_domains)
                    if result is not None:
                        return result
                
                del assignment[var]
                self.backtracks += 1
        
        return None


# Test AC-3
print("Backtracking with AC-3 Constraint Propagation")
print("=" * 60)

solver_ac3 = BacktrackingWithAC3(australia)
start = time.time()
solution = solver_ac3.solve()
elapsed = time.time() - start

if solution:
    print("Solution:")
    for region in sorted(solution.keys()):
        print(f"  {region}: {solution[region]}")
    print()
    print(f"Statistics:")
    print(f"  Assignments tried: {solver_ac3.assignments_tried}")
    print(f"  Backtracks: {solver_ac3.backtracks}")
    print(f"  Time: {elapsed*1000:.2f} ms")
    print()
    print("AC-3 reduced domain sizes before search!")

## Part 5: Sudoku Solver

### Sudoku as a CSP

**Variables**: 81 cells (9×9 grid)
**Domains**: {1, 2, 3, 4, 5, 6, 7, 8, 9} for each cell
**Constraints**:
- All cells in same row have different values
- All cells in same column have different values
- All cells in same 3×3 box have different values


In [ ]:
class Sudoku:
    """Sudoku puzzle solver using CSP."""
    
    def __init__(self, grid: np.ndarray):
        """
        Initialize Sudoku puzzle.
        
        Args:
            grid: 9x9 numpy array (0 for empty cells)
        """
        self.grid = grid.copy()
        self.size = 9
        self._build_csp()
    
    def _build_csp(self):
        """Build CSP from Sudoku grid."""
        # Variables: (row, col) for each cell
        self.variables = [(i, j) for i in range(9) for j in range(9)]
        
        # Domains: 1-9 for empty cells, fixed value for filled cells
        self.domains = {}
        for i, j in self.variables:
            if self.grid[i, j] == 0:
                self.domains[(i, j)] = list(range(1, 10))
            else:
                self.domains[(i, j)] = [int(self.grid[i, j])]
        
        # Build constraints (neighbors)
        self.neighbors = {var: set() for var in self.variables}
        
        for i, j in self.variables:
            # Row neighbors
            for k in range(9):
                if k != j:
                    self.neighbors[(i, j)].add((i, k))
            
            # Column neighbors
            for k in range(9):
                if k != i:
                    self.neighbors[(i, j)].add((k, j))
            
            # Box neighbors
            box_row, box_col = 3 * (i // 3), 3 * (j // 3)
            for bi in range(box_row, box_row + 3):
                for bj in range(box_col, box_col + 3):
                    if (bi, bj) != (i, j):
                        self.neighbors[(i, j)].add((bi, bj))
    
    def is_consistent(self, var: Tuple[int, int], value: int, 
                     assignment: Dict) -> bool:
        """Check if value assignment is consistent."""
        for neighbor in self.neighbors[var]:
            if neighbor in assignment and assignment[neighbor] == value:
                return False
        return True
    
    def solve(self) -> Optional[np.ndarray]:
        """Solve Sudoku puzzle."""
        assignment = {}
        
        # Add fixed values to assignment
        for i, j in self.variables:
            if self.grid[i, j] != 0:
                assignment[(i, j)] = int(self.grid[i, j])
        
        result = self._backtrack(assignment)
        
        if result:
            # Convert to grid
            solution = np.zeros((9, 9), dtype=int)
            for (i, j), value in result.items():
                solution[i, j] = value
            return solution
        return None
    
    def _backtrack(self, assignment: Dict) -> Optional[Dict]:
        """Backtracking with MRV heuristic."""
        if len(assignment) == len(self.variables):
            return assignment
        
        # MRV: select variable with smallest domain
        unassigned = [v for v in self.variables if v not in assignment]
        
        def count_legal_values(var):
            count = 0
            for value in self.domains[var]:
                if self.is_consistent(var, value, assignment):
                    count += 1
            return count
        
        var = min(unassigned, key=count_legal_values)
        
        for value in self.domains[var]:
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                
                result = self._backtrack(assignment)
                if result is not None:
                    return result
                
                del assignment[var]
        
        return None
    
    def visualize(self, grid: np.ndarray, title: str = "Sudoku"):
        """Visualize Sudoku grid."""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Draw grid
        for i in range(10):
            linewidth = 3 if i % 3 == 0 else 1
            ax.axhline(i, color='black', linewidth=linewidth)
            ax.axvline(i, color='black', linewidth=linewidth)
        
        # Fill numbers
        for i in range(9):
            for j in range(9):
                if grid[i, j] != 0:
                    # Check if it was given (in original puzzle)
                    if self.grid[i, j] != 0:
                        color = 'black'
                        weight = 'bold'
                    else:
                        color = 'blue'
                        weight = 'normal'
                    
                    ax.text(j + 0.5, 8.5 - i, str(int(grid[i, j])),
                           ha='center', va='center', fontsize=20,
                           color=color, fontweight=weight)
        
        ax.set_xlim(0, 9)
        ax.set_ylim(0, 9)
        ax.set_aspect('equal')
        ax.axis('off')
        ax.set_title(title, fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()


# Example Sudoku puzzle (easy)
print("Sudoku Solver")
print("=" * 60)

puzzle = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

sudoku = Sudoku(puzzle)
print("Puzzle:")
sudoku.visualize(puzzle, "Sudoku Puzzle")

# Solve
start = time.time()
solution = sudoku.solve()
elapsed = time.time() - start

if solution is not None:
    print(f"\n✓ Solved in {elapsed*1000:.2f} ms")
    sudoku.visualize(solution, "Sudoku Solution")
else:
    print("\n✗ No solution exists")

## Exercises

### Exercise 1: N-Queens as CSP
Formulate and solve the N-Queens problem as a CSP.
Compare with the local search solution from Lab 2.

In [ ]:
# TODO: Implement N-Queens as CSP
# Your code here
pass

### Exercise 2: Cryptarithmetic Puzzles
Solve SEND + MORE = MONEY where each letter is a unique digit.
Formulate as CSP and solve with backtracking.

In [ ]:
# TODO: Solve cryptarithmetic puzzle
# Your code here
pass

### Exercise 3: Course Scheduling
Create a course scheduling CSP with:
- 5 courses, 3 time slots, 2 rooms
- Some courses can't be at same time (student conflicts)
- Some professors can't teach at certain times

In [ ]:
# TODO: Implement course scheduling CSP
# Your code here
pass

## Summary

### Key Takeaways

1. **CSP Framework** - Variables, domains, constraints
2. **Backtracking** - Systematic search with pruning
3. **Heuristics** - MRV and LCV dramatically improve efficiency
4. **Arc Consistency** - Prune domains before and during search
5. **Real Applications** - Map coloring, Sudoku, scheduling

### When to Use CSPs

**Good fit**:
- Well-defined variables and constraints
- Need to find any solution (or all solutions)
- Need proof of infeasibility
- Combinatorial problems

**Not ideal**:
- Continuous optimization
- Need optimal solution (not just feasible)
- Constraints are soft/preferences

### Performance Tips

1. **Use MRV** - Fail fast on impossible problems
2. **Use LCV** - Preserve flexibility
3. **Use AC-3** - Reduce domain sizes early
4. **Good representation** - Choose variables wisely

### Next Steps

In Lab 4, we'll learn about **Genetic Algorithms** - evolutionary optimization for complex problems!
